# WRF-Python Tutorial 2017

## Bill Ladwig
## NCAR/CISL/VAST

# Topics

1. Introduction to jupyter notebooks, numpy, xarray
2. Overview of WRF-ARW Output Data
3. wrf-python
4. Plotting
5. Advanced

# 1.0 Introduction to jupyter, numpy, xarray

## What is jupyter notebook?

- The Jupyter Notebook is an open-source web application that allows you to create and share documents that contain live code, equations, visualizations and explanatory text. 

- The Jupyter Notebook actually consists of a document and an application

- The Jupyter Notebook application is a web browser application that allows editing and executing of jupyter notebook documents.


- Jupyter Notebook documents (usually ending with a .ipynb extension), are really just JSON-formatted text files that contain the code and rich text elements that will be rendered by the jupyter notebook application.  

- Jupyter notebook documents are NOT Python scripts, so do not try to run them via the 'python' command.  They need to be converted first.

- For this tutorial, when we refer to jupyter notebook, we're referring to both the application and document.

## Starting jupyter notebook

If you followed the installation instructions on the web, you should have created a "tutorial_2017" conda environment.  

To activate it, first open a terminal and type in:

```
source activate tutorial_2017
```

Raise your hand if you need help.

## Starting jupyter notebook

Now go to the directory where you stored the tutorial files (e.g. ~/wrf_python_tutorial) and execute the command "jupyter notebook"

```

cd ~/wrf_python_tutorial

jupyter notebook

```

If for some reason the browser does not launch automatically (Mac Sierra Users), open a web browser and go to:

```
http://localhost:8888/tree
```

Your web browser should look similar to this:
 
![alt](images/jupyter_home.png)


Now click on the "wrf_python_student.ipynb" link (just click the text).  

This should open a new browser tab that looks like:

![alt](images/jupyter_workbook.png)


# Cells

- A jupyter notebook is a collection of cells, similar to Mathematica.

- Cells can be either executable code or text (markdown).

- Cells can also be specified as slides, which is how this slide show was made (along with the Rise plugin)

## Executing Cells

1. Click on the desired cell.
2. Press **CTRL + RETURN** to execute the cell or press **SHIFT + RETURN** to execute the cell and advance to the next cell.
3. Alternatively, you can use the Cell dropdown menu

## Restarting the Notebook

If your notebook crashes for some reason:

1. Use the Kernel dropdown menu at the top.
2. Execute Kernel -> Restart & Clear Output.


## Shutting down the notebook

1. On your web browser, select the Home tab.
2. Click the check box next to wrf_python_student.ipynb.
3. Click the Shutdown button that will become available after step 2.
4. Now go to the terminal window where you typed in "jupyter notebook".
5. With the terminal window active, press **CTRL + C**.

# 1.1 Verifying your Jupyter Environment

To set up the tutorial to work with your files, modify the WRF_DIRECTORY and WRF_FILES variables to point to your WRF files.

**IMPORTANT**:  If for some reason your workbook crashes, you need to run this cell again before running the later examples.

In [ ]:
from __future__ import print_function

# This jupyter notebook command inserts matplotlib graphics in 
# to the workbook
%matplotlib inline

# Modify these to point to your own files
WRF_DIRECTORY = "./"
WRF_FILES = ["wrfout_d01_2005-08-28_00:00:00",
             "wrfout_d01_2005-08-28_12:00:00"]


# Do not modify the code below this line
#------------------------------------------------------
# Make sure the environment is good
from netCDF4 import Dataset
from xarray import DataArray
from wrf import (getvar, interplevel, vertcross, 
                 vinterp, ALL_TIMES)
import os

_WRF_FILES = [os.path.abspath(os.path.expanduser(
    os.path.join(WRF_DIRECTORY, f))) for f in WRF_FILES]

# Check that the WRF files exist
for f in _WRF_FILES:
    if not os.path.exists(f):
        raise ValueError("{} does not exist. "
            "Check for typos or incorrect directory.".format(full_path))

# Create functions so that the WRF files only need
# to be specified using the WRF_FILES global above
def single_wrf_file():
    global _WRF_FILES
    return _WRF_FILES[0]

def multiple_wrf_files():
    global _WRF_FILES
    return _WRF_FILES

print ("All tests passed!")
    


# 2.0 Overview of WRF Output Data

The first rule of data processing:

**"ALWAYS LOOK AT YOUR DATA"**

\- D. Shea

WRF can be configured in various ways and can have variables turned on and off.  If you run in to problems, it could be due to a variable missing.  If your plot doesn't look right, there could be a map projection issue.

Let's look at some WRF data...how do we do that?

There are numerous tools available to examine NetCDF data, from both outside and inside of Python.

- **ncdump** (used for this example)
- ncl_filedump
- netcdf4-python 
- xarray


## ncdump

ncdump is a program included with the NetCDF libraries that can be used to examine NetCDF data.

By supplying the '-h' option, only the data descriptions are returned.  Otherwise, you'll get all of the data values, which can span miles.

To run:

```
$ ncdump -h wrfout_d01_2005-08-28_00:00:00
```

<div id="nc_dims" style="font-size:75%;"/>
``` 
netcdf wrfout_d01_2005-08-28_00\:00\:00 {
dimensions:
    Time = UNLIMITED ; // (4 currently)
    
    DateStrLen = 19 ;
    
    west_east = 90 ;
    
    south_north = 73 ;
    
    bottom_top = 29 ;
    
    bottom_top_stag = 30 ;
    
    soil_layers_stag = 4 ;
    
    west_east_stag = 91 ;
    
    south_north_stag = 74 ;
    
```
</div>



<div id="nc_vars" style="font-size:75%;">
```
variables:
    char Times(Time, DateStrLen) ;
    float XLAT(Time, south_north, west_east) ;
        XLAT:FieldType = 104 ;
        XLAT:MemoryOrder = "XY " ;
        XLAT:description = "LATITUDE, SOUTH IS NEGATIVE" ;
        XLAT:units = "degree_north" ;
        XLAT:stagger = "" ;
        XLAT:coordinates = "XLONG XLAT" ;
    float XLONG(Time, south_north, west_east) ;
        XLONG:FieldType = 104 ;
        XLONG:MemoryOrder = "XY " ;
        XLONG:description = "LONGITUDE, WEST IS NEGATIVE" ;
        XLONG:units = "degree_east" ;
        XLONG:stagger = "" ;
        XLONG:coordinates = "XLONG XLAT" ;
        .
        .
        .
    float SST_INPUT(Time, south_north, west_east) ;
        SST_INPUT:FieldType = 104 ;
        SST_INPUT:MemoryOrder = "XY " ;
        SST_INPUT:description = "SEA SURFACE TEMPERATURE 
            FROM WRFLOWINPUT FILE" ;
        SST_INPUT:units = "K" ;
        SST_INPUT:stagger = "" ;
        SST_INPUT:coordinates = "XLONG XLAT XTIME" ;
```
</div>

<div id="nc_attrs" style="font-size:75%;">
```
// global attributes:
        :TITLE = " OUTPUT FROM WRF V3.7 MODEL" ;
        :START_DATE = "2005-08-28_00:00:00" ;
        :SIMULATION_START_DATE = "2005-08-28_00:00:00" ;
        :WEST-EAST_GRID_DIMENSION = 91 ;
        :SOUTH-NORTH_GRID_DIMENSION = 74 ;
        :BOTTOM-TOP_GRID_DIMENSION = 30 ;
        :DX = 30000.f ;
        :DY = 30000.f ;
        .
        .
        .
        :CEN_LAT = 28.00002f ;
        :CEN_LON = -89.f ;
        :TRUELAT1 = 30.f ;
        :TRUELAT2 = 60.f ;
        :MOAD_CEN_LAT = 28.00002f ;
        :STAND_LON = -89.f ;
        :POLE_LAT = 90.f ;
        :POLE_LON = 0.f ;
        :GMT = 0.f ;
        :JULYR = 2005 ;
        :JULDAY = 240 ;
        :MAP_PROJ = 1 ;
        :MAP_PROJ_CHAR = "Lambert Conformal" ;
        .
        .
        .
}
```
</div>

## Dimensions

WRF uses an Arakawa C-grid staggered grid [(taken from mmm website)] [1]

- Mass related quantities (pressure, temperature, etc) are computed at the center of a grid cell. 
- The u-component of the horizontal wind is calculated at the left and right edges of a grid cell.  It has one more 
  point in the x direction than the mass grid.
- The v-component of the horizontal wind is calculated at the bottom and top edges of a grid cell.  It has one more     point in the y direction than the mass grid. 
- The corners of each grid box are know as the 'staggered' grid, and has one additional point in both the x and y direction.

[1]: http://www2.mmm.ucar.edu/rt/amps/information/configuration/wrf_grid_structure.html

![alt](images/wrf_stagger.png)

<div id="nc_dims" style="font-size:75%;"/>
``` 
netcdf wrfout_d01_2005-08-28_00\:00\:00 {
dimensions:
    Time = UNLIMITED ; // (4 currently)
    
    DateStrLen = 19 ;
    
    west_east = 90 ;
    
    south_north = 73 ;
    
    bottom_top = 29 ;
    
    bottom_top_stag = 30 ; <-- Extra grid point
    
    soil_layers_stag = 4 ;
    
    west_east_stag = 91 ; <-- Extra grid point
    
    south_north_stag = 74 ; <-- Extra grid point
    
```
</div>

## Variables

- Each variable is made up of dimensions, attributes, and data values. 
- Pay special attention to the units and coordinates attribute.
  - The *coordinates* attribute specifies the variables that contain the latitude and longitude 
    information for each grid box (XLONG, XLAT).
  - If the domain is from a moving nest, then a time coordinate is also used (XTIME).
  - The coordinates are named in Fortran ordering, so they'll be listed in reverse.




<div id="var_example" style="font-size:75%;"/>
```
float P(Time, bottom_top, south_north, west_east) ; <- Dims
	P:FieldType = 104 ;                       <-  Attribute
	P:MemoryOrder = "XYZ" ;                   <-  Attribute
    P:description = "perturbation pressure" ; <-  Attribute
	P:units = "Pa" ;                          <-  Attribute
    P:stagger = "" ;                          <-  Attribute
	P:coordinates = "XLONG XLAT XTIME" ;      <-  Attribute
    
data:

 P =
  339.8281, 340.3281, 340.25, 341.4531, ... 
    355.8672, 356.9531, 361.2578, 365.7188,  ...

```
</div>

## Global Attributes

- Provide a description of how the model was set up (resolution, map projection, microphysics, etc)
- For plotting, the map projection parameters will be the most important.
- wrf-python uses this information to build the mapping object in your plotting system of choice - basemap, cartopy, pyngl.




<div id="proj_stuff" style="font-size:75%;"/>
```
.
.
.
:CEN_LAT = 28.00002f ;
:CEN_LON = -89.f ;
:TRUELAT1 = 30.f ;
:TRUELAT2 = 60.f ;
:MOAD_CEN_LAT = 28.00002f ;
:STAND_LON = -89.f ;
:POLE_LAT = 90.f ;
:POLE_LON = 0.f ;
.
.
:MAP_PROJ = 1 ;
:MAP_PROJ_CHAR = "Lambert Conformal" ;
.
.
.
```
</div>

## Your Turn!

### Example 2.1: Running ncdump

In [ ]:
from subprocess import Popen, PIPE, STDOUT

file_path = single_wrf_file()

# This simply executes 'ncdump -h {wrf_file}' 
# from Python
p = Popen(["ncdump", "-h", "{}".format(file_path)], 
          stdout=PIPE, stderr=STDOUT)
output, _ = p.communicate()

print (output)


## Reading a WRF File in Python

You have several options to read a WRF NetCDF file in Python. 

- **netcdf4-python**
- PyNIO (currently Python 2.x only)
- xarray (Dataset not natively supported yet in wrf-python)


## netcdf4-python Example

``` python
from netCDF4 import Dataset

file_path = "./wrfout_d01_2005-08-28_00:00:00"

wrf_file = Dataset(file_path)

```

## Your Turn!

### Example 2.2: Using netcdf4-python

In [ ]:
from netCDF4 import Dataset

file_path = single_wrf_file()

wrf_file = Dataset(file_path)

print(wrf_file)


## Getting Variables and Attributes

netcdf4-python uses an old API that was originally created for a package called Scientific.IO.NetCDF.

This is now in scipy.

PyNIO also uses this API.

xarray does not use this API.

Some of it may look a little dated.




### Getting global attributes

The get the full dictionary of global attributes, use the \_\_dict\_\_ attribute.  To work with one attribute at a time, you can use the getncattr and setncattr methods. 

``` python
global_attrs = wrf_file.__dict__

# To get the value for MAP_PROJ, you can do:
map_proj = wrf_file.__dict__["MAP_PROJ"]

# Or more cleanly
map_prof = wrf_file.getncattr("MAP_PROJ")

```

### Getting variables, variable attributes, and variable data

All variables are stored in a dictionary attribute called *variables*.  

Let's get the perturbation pressure ("P") variable.

``` python

# This will return a netCDF4.Variable object
p = wrf_file.variables["P"]

```

To get the variable attributes, you can use the \_\_dict\_\_ attribute to get a dictionary of all attributes, or the *getncattr* function if you already know the attribute name.

``` python
# Return a dictionary of all of P's 
# attributes
p_attrs = p.__dict__

# Let's just get the 'coordinates' attribute
p_coords = p.getncattr("coordinates")

```

To get the variable's data as a numpy array, you need to use Python's [ ] API (\_\_getitem\_\_ for those that are more familiar with Python's data model).  

``` python

# Get a numpy array for all times
p_all_data = p[:,:,:,:]

# In numpy, there is implicit expansion of ':'
# across all dimensions. So, this is the 
# same as p[:,:,:,:]
p_all_data = p[:]

# You can also request specific values 
# by supplying indexes.  This will 
# extract the numpy array for time 
# index 0.

p_t0_data = p[0,:]


```

## Your Turn!

### Example 2.3: Variables, Attributes, and Data with netcd4-python

In [ ]:
from netCDF4 import Dataset

file_path = single_wrf_file()

# Create the netCDF4.Dataset object
wrf_file = Dataset(file_path)

# Get the global attribute dict
global_attrs = wrf_file.__dict__
print ("Global attributes for the file")
print(global_attrs)
print ("\n")

# Just get the 'MAP_PROJ' attribute
map_proj = wrf_file.getncattr("MAP_PROJ")
print ("The MAP_PROJ attribute:")
print (map_proj)
print("\n")

# Get the perturbation pressure variable
p = wrf_file.variables["P"]
print ("The P variable: ")
print(p)
print ("\n")

# Get the P attributes
p_attrs = p.__dict__
print ("The attribute dict for P")
print (p_attrs)
print ("\n")

# Get the 'coordinates' attribute for P
coords = p.getncattr("coordinates")
print ("Coordinates for P:")
print (coords)
print ("\n")

# Get the P numpy array for all times
p_all_data = p[:]
print ("The P numpy array: ")
print (p_all_data)
print ("\n")

# Get the P numpy array for time 0
p_t0_data = p[0,:]
print ("P array at time 0:")
print (p_t0_data)
print ("\n")

## Pop Quiz

What is the first rule of data processing?

    A) YOU DO NOT TALK ABOUT DATA PROCESSING
    B) 60% OF THE TIME, IT WORKS EVERY TIME
    C) ALWAYS LOOK AT YOUR DATA

# 3.0 wrf-python

wrf-python provides functionality similar to what is found in the NCL-WRF package:

- over 30 diagnostics calculations
- several interpolation routines (horizontal level, vertical cross section, horizontal "surface")
- plot helper utilities for cartopy, basemap, and PyNGL


The most commonly used functions:

- **getvar**: Extracts variables and diagnostic variables
- **interplevel**: Interpolates a variable to a horizontal plane at a specified level
- **vertcross**: Interpolates a 3D variable to a vertical cross section
- **vinterp**: Interpolates a variable to a new surface (e.g. theta-e)

## The *getvar* function

The *getvar* function can be used to:

- Extract NetCDF variables from a file, similar to netcdf4-python or PyNIO.
- Compute diagnostic variables.
- Concatenate a variable (either NetCDF or diagnostic) across multiple files. 



### Simple *getvar* Example for HGT

``` python

from netCDF4 import Dataset
from wrf import getvar

file_path = "./wrfout_d01_2005-08-28_00:00:00"

wrf_file = Dataset(file_path)

hgt = getvar(wrf_file, "HGT", timeidx=0)

```

## Your Turn!

### Example 3.1: Using *getvar* to Extract a WRF NetCDF Variable

In [ ]:
from netCDF4 import Dataset
from wrf import getvar

file_path = single_wrf_file()

wrf_file = Dataset(file_path)

hgt = getvar(wrf_file, "HGT", timeidx=0)

print(hgt)

## Computing a Diagnostic Variable with *getvar*

A list of available diagnostics is available at: http://wrf-python.readthedocs.io/en/latest/diagnostics.html

In this example, we're going to compute sea level pressure.  

Note the 'units' keyword argument.  Some diagnostics support several choices for units.  However, unit support is still relatively primitive.  

``` python

from netCDF4 import Dataset
from wrf import getvar

file_path = "./wrfout_d01_2005-08-28_00:00:00"

wrf_file = Dataset(file_path)

slp = getvar(wrf_file, "slp", 
             timeidx=0, units="hPa")


```

## Your Turn!

### Example 3.2: Using *getvar* to compte Sea Level Pressure (SLP)

Also try changing the units for by specifiying the following values:  'hPa', 'Pa', 'atm', 'mmhg'

In [ ]:
from netCDF4 import Dataset
from wrf import getvar

file_path = single_wrf_file()

wrf_file = Dataset(file_path)

slp = getvar(wrf_file, "slp", timeidx=0, units="mmhg")

print (slp)

## Combining Across Mulitple Files

wrf-python has two methods for combining a variable across multiple files

- **cat** - combines the the variable along the Time dimension (Note: you must put the files in the order you want them)
- **join** - creates a new left-most dimension for each file


To extract all times in to a single array, set *timeidx* to wrf.ALL_TIMES (an alias for None).  

In this example, we're using the 'cat' method, which is the most common.

``` python
from netCDF4 import Dataset
from wrf import getvar, ALL_TIMES

file_paths = [
     "./wrfout_d01_2005-08-28_00:00:00",
     "./wrfout_d01_2005-08-28_12:00:00"
     ]

wrf_files = [Dataset(file_paths[0]),
             Dataset(file_paths[1])]

slp = getvar(wrf_files, "slp", 
             timeidx=ALL_TIMES,
             method="cat")

```

## Your Turn!


### Example 3.3: Combining Files Using the 'cat' Method

In [ ]:
from netCDF4 import Dataset
from wrf import getvar, ALL_TIMES

file_paths = multiple_wrf_files()

wrf_files = [Dataset(file_paths[0]),
             Dataset(file_paths[1])]

slp = getvar(wrf_files, "slp", timeidx=ALL_TIMES, method="cat")

print (slp)

### Example 3.4: Combining Files Using the 'join' Method

In [ ]:
from netCDF4 import Dataset
from wrf import getvar, ALL_TIMES

file_paths = multiple_wrf_files()

wrf_files = [Dataset(file_paths[0]),
             Dataset(file_paths[1])]

slp = getvar(wrf_files, "slp", timeidx=ALL_TIMES, method="join")

print (slp)

## Interpolation Routines

- **interplevel** - linear interpolation to a horizontal plane at a specified height or pressure level
- **vertcross** - vertical cross section interpolation to a vertical plane through two specified points 
  (or a pivot point and angle)
- **vinterp** - interpolates to a "surface", which could be pressure levels or temperature levels like theta-e

## The *interplevel* function

- The easiest way to get a field at a specified height or pressure vertical level (500 mb, 5000 m, etc)
- Uses linear interpolation, which is fastest and generally "good enough" for plotting
- You should use *vinterp* if you want more accuracy, since the interpolation is done with the decaying exponential 
  pressure profile

### *interplevel* Example
 
Let's get the 500 hPa geopotential height in decameters
 
``` python
 
from netCDF4 import Dataset
from wrf import getvar, interplevel

file_path = "./wrfout_d01_2005-08-28_00:00:00"

wrf_file = Dataset(file_path)

pres = getvar(wrf_file, "pressure", timeidx=0)

ht = getvar(wrf_file, "z", timeidx=0, 
            units="dm")

ht_500 = interplevel(ht, pres, 500.0)
 
```

## Your Turn!

### Example 3.5: Interpolate to 500 hPa Using *interplevel*

In [ ]:
from netCDF4 import Dataset
from wrf import getvar, interplevel

file_path = single_wrf_file()

wrf_file = Dataset(file_path)

pres = getvar(wrf_file, "pressure", timeidx=0)
ht = getvar(wrf_file, "z", timeidx=0, units="dm")

ht_500 = interplevel(ht, pres, 500.0)

print (ht_500)

## The *vertcross* function

Vertical cross sections can be confusing to people.  The idea is to draw a horizontal line at the surface, and the cross section is defined as a vertical plane extending up from this line.

- The new 'x-axis' in the cross section is the points along the line you made.  The line can be defined by:
  1. defining a start point and an end point by using (x,y) grid coordinates or (latitude, longitude) coordinates.
  2. defining a pivot point and an angle, which is useful for cross sections that will span most of the domain.
- The new 'y-axis' will be a set of vertical levels chosen to be predefined levels, or you can choose them.  

Also important to note that the horizontal line distance is discretized in to integer points, so the final point on the line may be slightly different than the end_point you have chosen.





### Introducing the *CoordPair* class

A *CoordPair* is simply used to store (x,y) coordinates, or (lat,lon) coordinates.  It is also possible to have (x, y, lat, lon), but that's rarely used.  

The *CoordPair* will be used to define your cross section line.

``` python
from wrf import CoordPair

# Creating an x,y pair
x_y_pair = CoordPair(x=10, y=20)

# Creating a lat,lon pair
lat_lon_pair = CoordPair(lat=30.0, lon=-120.0)

```

### *vertcross* Example

In this example, we're going to define the cross section using a start point and and end point.  

We're going to let the algorithm pick the levels, which are at ~1% increments.

``` python
 
from netCDF4 import Dataset
from wrf import getvar, vertcross, CoordPair

file_path = "./wrfout_d01_2005-08-28_00:00:00"
wrf_file = Dataset(file_path)

# Making a diagonal cross section line from 
# bottom left to top right.
bottom_left = CoordPair(x=0, y=0)

top_right = CoordPair(x=-1, y=-1)

# Let's get wind speed in kts
wspd_wdir = getvar(wrf_file, "wspd_wdir", 
                   timeidx=0, units="kt")
                   
wspd = wspd_wdir[0,...]

# Get the height levels
ht = getvar(wrf_file, "z", timeidx=0)

wspd_cross = vertcross(wspd, ht, 
               start_point=bottom_left, 
               end_point=top_right)
               
```

## Your Turn!

### Example 3.6: Interpolate to a Vertical Cross Section with *vertcross*

In [ ]:
from netCDF4 import Dataset
from wrf import getvar, vertcross, CoordPair

file_path = single_wrf_file()
wrf_file = Dataset(file_path)

# Making a diagonal cross section line from 
# bottom left to top right.
bottom_left = CoordPair(x=0, y=0)

top_right = CoordPair(x=-1, y=-1)

# Let's get wind speed in kts
wspd_wdir = getvar(wrf_file, "wspd_wdir", 
                   timeidx=0, units="kt")
                   
wspd = wspd_wdir[0,:]

# Get the height levels
ht = getvar(wrf_file, "z", timeidx=0)

wspd_cross = vertcross(wspd, ht, 
               start_point=bottom_left, 
               end_point=top_right)

print (wspd_cross)


## The *vinterp* function

- Used for interpolating a field to a type of surface:
  - pressure
  - geopotential height
  - theta
  - theta-e
- User must specify the interpolation level(s) on the new surface
- A smarter, albeit slower and more complicated, version of *interplevel*

### *vinterp* Example

In this example, we're going to interpolate pressure to theta-e levels (good for doing isentropic analysis).  

``` python

from netCDF4 import Dataset
from wrf import getvar, vinterp 

file_path = "./wrfout_d01_2005-08-28_00:00:00"
wrf_file = Dataset(file_path) 

pres = getvar(wrf_file, "pressure", timeidx=0)

# Interpolate pressure to theta-e levels                 
interp_levels = [280, 285, 290, 292, 294, 
                 296, 298, 300, 305, 310]

pres_eth = vinterp(wrf_file, 
               field=pres, 
               vert_coord="theta-e", 
               interp_levels=interp_levels, 
               extrapolate=False, 
               field_type="pressure", 
               log_p=False)
```

## Your Turn!

### Example 3.7: Interpolate to Theta-e Levels with *vinterp*

In [ ]:
from netCDF4 import Dataset
from wrf import getvar, vinterp 

file_path = "./wrfout_d01_2005-08-28_00:00:00"
wrf_file = Dataset(file_path) 

pres = getvar(wrf_file, "pressure", timeidx=0)

# Interpolate pressure to theta-e levels                 
interp_levels = [280., 285., 290., 292., 294., 
                 296., 298., 300., 305., 310.]

pres_eth = vinterp(wrf_file, 
               field=pres, 
               vert_coord="theta-e", 
               interp_levels=interp_levels, 
               extrapolate=True, 
               field_type="pressure", 
               log_p=False,
               timeidx=0)

print (pres_eth)

# Other Useful Functions

## *to_np*

Converts an xarray.DataArray to a numpy array.

This is often necessary when passing wrf-python variables to the plotting functions or to a compiled extension module.

This routines does the following:

- If no missing values, then it simply calls the *xarray.DataArray.values* property.
- If missing values are found, then it replaces the NaN that xarray uses for missing 
  values with the fill value found in the attributes.


### *to_np* Example

``` python

from netCDF4 import Dataset
from wrf import getvar, to_np 

file_path = "./wrfout_d01_2005-08-28_00:00:00"
wrf_file = Dataset(file_path) 

pres_xarray = getvar(wrf_file, "pressure", timeidx=0)

pres_numpy = to_np(pres_xarray)

```

## xy_to_ll and ll_to_xy

These routines convert from grid (x,y) coordinates to (lat,lon) coordinates.  

Works with a single point or sequences of points.

### xy_to_ll and ll_to_xy Example

``` python

from netCDF4 import Dataset
from wrf import getvar, xy_to_ll, ll_to_xy 

file_path = "./wrfout_d01_2005-08-28_00:00:00"
wrf_file = Dataset(file_path)

lat_lon = xy_to_ll(wrf_file, 
                  [20, 30], 
                  [50,75])

x_y = ll_to_xy(wrf_file, 
               lat_lon[0,:],
               lat_lon[1,:])


```

## Your Turn!

### Example 3.8: *xy_to_ll* and *ll_to_xy*

In [ ]:
from netCDF4 import Dataset
from wrf import getvar, xy_to_ll, ll_to_xy 

file_path = single_wrf_file()
wrf_file = Dataset(file_path)

lat_lon = xy_to_ll(wrf_file, [20, 30], [50,75])

print ("lat,lon values")
print (lat_lon)
print ("\n")

x_y = ll_to_xy(wrf_file, lat_lon[0,:], lat_lon[1,:])

print ("x,y values")
print (x_y)


# 4.0 Plotting

Plotting wrf-python variables in Python can be done using either matplotlib or PyNGL.

For this tutorial, we're going to focus on matplotlib (using cartopy for the mapping).

We're going to use matplotlib 1.5.3, since cartopy is still having some issues with matplotlib 2.x.

## Extremely Brief Overview of Matplotlib

Under the hood, matplotlib uses and object oriented API.

In simplest terms, a matplotlib plot consists of Figure object that contains an Axes object (or multiple Axes objects).

The Axes object is where the action is.

The Axes object contains the methods for contouring, making histograms, adding colorbar, etc.  





### The pyplot API

Most matplotlib users do no need to know much about the underlying object oriented API.

Matplotlib includes a series of standalone functions that wrap around the object oriented API.

These standalone functions are in the matplotlib.pyplot package and it was designed to look similar to the Matlab API.

## Your Turn!

### Example 4.1: Single Wind Barb Example with pyplot

In this example, we are going to plot a single wind barb in the center of the domain using the pyplot API.

In [ ]:
from matplotlib import pyplot
import numpy as np

# Make a 5x5 grid of missing u,v values
u = np.ma.masked_equal(np.zeros((5,5)), 0)
v = np.ma.masked_equal(np.zeros((5,5)), 0)

# Add u,v winds to center of domain
u[2,2] = 10.0
v[2,2] = 10.0

# Draw a single wind barb in the middle using pyplot API
# Note:  the axes objects are "hidden" in these functions
fig = pyplot.figure()
pyplot.barbs(u, v)

# Set the x and y ranges so the barb is in the middle
pyplot.xlim(0, 4)
pyplot.ylim(0, 4)

pyplot.show()




Often you will find yourself mixing the object oriented API with the pyplot API.  

This is required when making subplots, but that is beyond the scope of this tutorial.  

The next example shows how to make the single wind barb using the axes object directly.

## Your Turn!

### Example 4.2: Single Wind Barb Using the Axes Object

In [ ]:
from matplotlib import pyplot
import numpy as np

# Make a 5x5 grid of missing u,v values
u = np.ma.masked_equal(np.zeros((5,5)), 0)
v = np.ma.masked_equal(np.zeros((5,5)), 0)

# Add u,v winds to center of domain
u[2,2] = 10.0
v[2,2] = 10.0

# We'll use pyplot to create the figure and 
# get the axes
fig = pyplot.figure()
ax = pyplot.axes() # <- Remember this line

# Now use the axes directly to create the barbs
ax.barbs(u, v)

# Set the x and y ranges using the axes directly
ax.set_xlim(0, 4)
ax.set_ylim(0, 4)

pyplot.show()

## wrf-python Plotting Helper Functions

wrf-python has several functions to help with plotting when using cartopy, basemap, or PyNGL.  

- **get_cartopy, get_basemap, get_pyngl**: Returns the mapping object used by the plotting system
- **latlon_coords**: Returns the latitude and longitude coordinate variables
- **get_bounds**: Returns the geographic boundaries for the variable

## Plotting with cartopy

Cartopy uses the same API as matplotlib by returning a matplotlib.axes.Axes subclass (cartopy.mpl.geoaxes.GeoAxes) when a *projection* keyword argument is passed to the matplotlib.pyplot.axes function.  

### Getting the GeoAxes object

``` python
import matplotlib.pyplot
import cartopy.crs

# Set up a standard map for latlon data.
geo_axes = pyplot.axes(projection=cartopy.crs.PlateCarree())

```

### Getting the cartopy projection object using wrf-python

When xarray is installed and enabled, wrf-python carries the projection information around in the metadata of a variable.  

You can use the *get_cartopy* function to extract the cartopy projection object from a variable.  

``` python

from netCDF4 import Dataset
from wrf import getvar, get_cartopy

file_path = "./wrfout_d01_2005-08-28_00:00:00"

wrf_file = Dataset(file_path)

terrain = getvar(wrf_file, "ter", timeidx=0)

cart_proj = get_cartopy(terrain)

```

## Your Turn!

### Example 4.3: Making a Plot of Terrain

Let's make a plot of terrain.  It's the easiest way to check if your map is correct.

In [ ]:
import numpy
from matplotlib import pyplot
from matplotlib.cm import get_cmap
from cartopy import crs
from cartopy.feature import NaturalEarthFeature
from netCDF4 import Dataset
from wrf import getvar, to_np, get_cartopy, latlon_coords

file_path = single_wrf_file()
wrf_file = Dataset(file_path)

# Get the terrain height
terrain = getvar(wrf_file, "ter", timeidx=0)

# Get the cartopy object and the lat,lon coords
cart_proj = get_cartopy(terrain)
lats, lons = latlon_coords(terrain)

# Create a figure and get the GetAxes object
fig = pyplot.figure(figsize=(10, 7.5))
geo_axes = pyplot.axes(projection=cart_proj)

# Download and add the states and coastlines
# See the cartopy documentation for more on this.
states = NaturalEarthFeature(category='cultural', 
                             scale='50m', 
                             facecolor='none',
                             name='admin_1_states_provinces_shp')
geo_axes.add_feature(states, linewidth=.5)
geo_axes.coastlines('50m', linewidth=0.8)

# Set the contour levels
levels = numpy.arange(250., 5000., 250.)

# Make the contour lines and fill them.
pyplot.contour(to_np(lons), to_np(lats), 
               to_np(terrain), levels=levels, 
               colors="black",
               transform=crs.PlateCarree())
pyplot.contourf(to_np(lons), to_np(lats), 
                to_np(terrain), levels=levels,
                transform=crs.PlateCarree(),
                cmap=get_cmap("terrain"))
             
# Add a color bar. The shrink often needs to be set 
# by trial and error.
pyplot.colorbar(ax=geo_axes, shrink=.86)

pyplot.show()

## Cropping

Sometimes WRF domains are much larger than what you care about.

Plots can be cropped in two ways using wrf-python:

1. Crop the data before plotting.
  - Less data to process = faster!
  - There's a slight risk of issues at borders, 
    but matplotlib seems pretty smart about this.
2. Crop the domain with matplotlib using x,y axis limits.
  - Runs slower since all of the domain is contoured.
  - Should always be correct.

### Method 1: Cropping then Plotting

If you are cropping the data, then cartopy should just work without worrying about setting the axis limits, as long as xarray is installed and enabled.

Let's start by taking a quick look at sea level pressure.


## Your Turn!

### Example 4.4: Full Plot of Sea Level Pressure

In [ ]:
import numpy
from matplotlib import pyplot
from matplotlib.cm import get_cmap
from cartopy import crs
from cartopy.feature import NaturalEarthFeature
from netCDF4 import Dataset
from wrf import getvar, to_np, get_cartopy, latlon_coords

file_path = single_wrf_file()
wrf_file = Dataset(file_path)

# Get the terrain height
slp = getvar(wrf_file, "slp", timeidx=0)

# Get the cartopy object and the lat,lon coords
cart_proj = get_cartopy(slp)
lats, lons = latlon_coords(slp)

# Create a figure and get the GetAxes object
fig = pyplot.figure(figsize=(10, 7.5))
geo_axes = pyplot.axes(projection=cart_proj)

# Download and add the states and coastlines
# See the cartopy documentation for more on this.
states = NaturalEarthFeature(category='cultural', 
                             scale='50m', 
                             facecolor='none',
                             name='admin_1_states_provinces_shp')
geo_axes.add_feature(states, linewidth=.5)
geo_axes.coastlines('50m', linewidth=0.8)

# Set the contour levels so that all plots match
levels = numpy.arange(980.,1030.,2.5)

# Make the contour lines and fill them.
pyplot.contour(to_np(lons), to_np(lats), 
               to_np(slp), levels=levels, colors="black",
               transform=crs.PlateCarree())
pyplot.contourf(to_np(lons), to_np(lats), 
                to_np(slp), levels=levels, 
                transform=crs.PlateCarree(),
                cmap=get_cmap("jet"))
             
# Add a color bar. The shrink often needs to be set 
# by trial and error.
pyplot.colorbar(ax=geo_axes, shrink=.86)

pyplot.show()

## Your Turn!

### Example 4.5: Cropping by Slicing the Data

Let's crop the data to the lower right quadrant.

In [ ]:
import numpy
from matplotlib import pyplot
from matplotlib.cm import get_cmap
from cartopy import crs
from cartopy.feature import NaturalEarthFeature
from netCDF4 import Dataset
from wrf import getvar, to_np, get_cartopy, latlon_coords

file_path = single_wrf_file()
wrf_file = Dataset(file_path)

# Get the terrain height
slp = getvar(wrf_file, "slp", timeidx=0)

# Determine the center of the domain in grid coordinates
slp_shape = slp.shape
center_y = int(slp_shape[-2]/2.) - 1
center_x = int(slp_shape[-1]/2.) - 1

# Slice from bottom to middle for y
# Slice from middle to right for x
slp_quad = slp[..., 0:center_y+1, center_x:]

# Get the cartopy object and the lat,lon coords
cart_proj = get_cartopy(slp_quad)
lats, lons = latlon_coords(slp_quad)

# Create a figure and get the GetAxes object
fig = pyplot.figure(figsize=(10, 7.5))
geo_axes = pyplot.axes(projection=cart_proj)

# Download and add the states and coastlines
# See the cartopy documentation for more on this.
states = NaturalEarthFeature(category='cultural', 
                             scale='50m', 
                             facecolor='none',
                             name='admin_1_states_provinces_shp')
geo_axes.add_feature(states, linewidth=.5)
geo_axes.coastlines('50m', linewidth=0.8)

# Set the contour levels so that all plots match
levels = numpy.arange(980.,1030.,2.5)

# Make the contour lines and fill them.
pyplot.contour(to_np(lons), to_np(lats), 
               to_np(slp_quad), levels=levels, colors="black",
               transform=crs.PlateCarree())
pyplot.contourf(to_np(lons), to_np(lats), 
                to_np(slp_quad), levels=levels, 
                transform=crs.PlateCarree(),
                cmap=get_cmap("jet"))
             
# Add a color bar. The shrink often needs to be set 
# by trial and error.
pyplot.colorbar(ax=geo_axes, shrink=.83)

pyplot.show()

### Method 2: Cropping by Setting x and y Extents

This time, let's crop the domain by using the x and y extents in matplotlib.

Also, we're going to crop the domain using lat,lon geographic boundaries.

#### Introducing the GeoBounds class

To create geographic boundaries, you supply a GeoBounds object constructed with set of bottom_left and top_right CoordPair objects.

The CoordPair objects need to use the *lat* and *lon* arguments.

``` python

from wrf import CoordPair, GeoBounds

bottom_left = CoordPair(lat=29.5, lon=-110)
top_right = CoordPair(lat=30.0, lon=-109.3)

geo_bounds = GeoBounds(bottom_left, top_right)

```

### Setting the Cartopy Extents

After setting up the GeoBounds objects, you can use *cartopy_xlim* and *cartopy_ylim* functions to set the extents.  

Note: Cartopy also has an API for doing this, but it doesn't work correctly for some projections like the RotatedPole.  

``` python

from wrf import (CoordPair, GeoBounds, getvar, 
                 cartopy_xlim, cartopy_ylim)

bottom_left = CoordPair(lat=29.5, lon=-110)
top_right = CoordPair(lat=30.0, lon=-109.3)

geo_bounds = GeoBounds(bottom_left, top_right)
.
. (Set up variable, figure, geo_axes, etc)
.

xlim = cartopy_xlim(variable, 
                    geobounds=geo_bounds))
                    
geo_axes.set_xlim(xlim)
         
ylim = cartopy_ylim(variable,
                    geobounds=geo_bounds))
                    
geo_axes.set_ylim(ylim)


```

## Your Turn!

### Example 4.6: Cropping by Setting the X,Y Extents

In [ ]:
import numpy
from matplotlib import pyplot
from matplotlib.cm import get_cmap
from cartopy import crs
from cartopy.feature import NaturalEarthFeature
from netCDF4 import Dataset
from wrf import getvar, to_np, get_cartopy, latlon_coords
from wrf import xy_to_ll, cartopy_xlim, cartopy_ylim
from wrf import CoordPair, GeoBounds

file_path = single_wrf_file()
wrf_file = Dataset(file_path)

# Get the terrain height
slp = getvar(wrf_file, "slp", timeidx=0)

# Get the cartopy object and the lat,lon coords
cart_proj = get_cartopy(slp)
lats, lons = latlon_coords(slp)

# Create a figure and get the GetAxes object
fig = pyplot.figure(figsize=(10, 7.5))
geo_axes = pyplot.axes(projection=cart_proj)

# Download and add the states and coastlines
# See the cartopy documentation for more on this.
states = NaturalEarthFeature(category='cultural', 
                             scale='50m', 
                             facecolor='none',
                             name='admin_1_states_provinces_shp')
geo_axes.add_feature(states, linewidth=.5)
geo_axes.coastlines('50m', linewidth=0.8)

# Set the contour levels so that all plots match
levels = numpy.arange(980.,1030.,2.5)

# Make the contour lines and fill them.
pyplot.contour(to_np(lons), to_np(lats), 
               to_np(slp), levels=levels, colors="black",
               transform=crs.PlateCarree())
pyplot.contourf(to_np(lons), to_np(lats), 
                to_np(slp), levels=levels, 
                transform=crs.PlateCarree(),
                cmap=get_cmap("jet"))
             
# Add a color bar. The shrink often needs to be set 
# by trial and error.
pyplot.colorbar(ax=geo_axes, shrink=.83)

# Set up the x, y extents

# Determine the center of the domain in grid coordinates
slp_shape = slp.shape
start_y = 0
center_y = int(slp_shape[-2]/2.) - 1
center_x = int(slp_shape[-1]/2.) - 1
end_x = int(slp_shape[-1]) - 1

# Get the lats and lons for the start, center, and end points
# (Normally you would just set these yourself)
center_latlon = xy_to_ll(wrf_file, 
                         [center_x, end_x], 
                         [start_y, center_y])

start_lat = center_latlon[0,0]
end_lat = center_latlon[0,1]
start_lon = center_latlon[1,0]
end_lon = center_latlon[1,1]

# Set the extents
geo_bounds = GeoBounds(CoordPair(lat=start_lat, lon=start_lon),
                       CoordPair(lat=end_lat, lon=end_lon))
geo_axes.set_xlim(cartopy_xlim(slp, geobounds=geo_bounds))
geo_axes.set_ylim(cartopy_ylim(slp, geobounds=geo_bounds))

pyplot.show()